In [55]:
%%capture
!pip install ibm-watson


In [56]:
%%capture
import warnings
warnings.filterwarnings('ignore')

!jupyter nbextension enable --py widgetsnbextension --sys-prefix
!jupyter serverextension enable voila --sys-prefix




In [57]:
from ibm_watson import DiscoveryV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from requests.auth import HTTPBasicAuth
from ibm_cloud_sdk_core.api_exception import ApiException


apikey = 'ZFd4ATmFd2zfGPcOIb14krOmYpX3yTEgum4jromwb1I1'
collection_id = 'd36fe302-d108-48be-8a07-87d7c0a958b3'
environment_id = '8aedd3e9-c7c1-4bc7-b9dd-4425ebbfc980'
url = 'https://api.eu-de.discovery.watson.cloud.ibm.com/instances/abd52fa4-a3a7-4f45-bba2-cfc8f08bc135'
version = '2019-04-30'


authenticator = IAMAuthenticator(apikey)
auth = HTTPBasicAuth('apikey', apikey)
discovery = DiscoveryV1(
    version=version,
    authenticator=authenticator
)

headers = {
    'content-type': "application/json"
    }
    
discovery.set_service_url(url)


In [58]:
import ipywidgets as widgets
import json
import csv
import requests


import numpy as np
import pandas as pd

In [59]:



titulo = widgets.HTML(value = '<h1>Chatbot</h1>')
titulo



HTML(value='<h1>Chatbot</h1>')

In [60]:
out = widgets.Output(layout={'border': '1px solid black'})
out




Output(layout=Layout(border='1px solid black'))

In [61]:
from ipywidgets import interact, widgets
from IPython.display import display, clear_output



sliders = list()
perguntas = list()
respostas = list()
botoes = list()
doc_ids = list()
colunas_notas = [f"Notas({x})" for x in range (0,5)]



text = widgets.Text(
    value='',
    placeholder='',
    description='',
    disabled=False
)
display(text)







def callback_botao(b):
    
    n_botao = int(b.tooltip)
    
    
    notas = [(slider.value) for slider in sliders[n_botao]]
    

    
    notas_dict = {k:v for k, v in zip(colunas_notas, notas)}
    #teste123 = {k:", ".join(value) for k,value in notas_dict.items()}  #converter para stirng
    
    
    
    
    dataframe = pd.read_csv("teste_relevancia.tsv", sep='\t')
    dataframe = dataframe.astype(str)
    
    
    
    if (perguntas[n_botao] not in dataframe.Perguntas.values):
        
        
        

        merged_dictionaries = {**{"Perguntas" : perguntas[n_botao]},**respostas[n_botao],**doc_ids[n_botao],**notas_dict} 


        dataframe = dataframe.append(merged_dictionaries, ignore_index= True)
        dataframe.to_csv("teste_relevancia.tsv", sep="\t",index = False)
        
    else:
        
         passou = True
        

                
#         #acrescentar mais uma nota para uma pergunta repetida 
         dataframe.loc[dataframe['Perguntas'] == perguntas[n_botao], notas_colunas] += [f',{x}' for x in notas]
        
        
         dataframe.to_csv("teste_relevancia.tsv", sep="\t",index = False)

        
    
    
    with out:
        print(f"notas_dict = {notas_dict}")
        display(dataframe)
        

def callback(wdgt):   # EVENT HANDLER APÓS O USUÁRIO ENVIAR A PERGUNTA
    
    with out: 
        print(wdgt.value)   #insere a pergunta no container do bate papo
        print('\n\n')
        texto = wdgt.value
        
        perguntas.append(texto)
        n_botao = len(perguntas) - 1
        wdgt.value = ''
        
        #chamada discovery
        result = discovery.query(environment_id=environment_id, collection_id=collection_id, deduplicate=False, highlight=True, passages=True, passages_count=5, natural_language_query= texto, count=5)
        
        
        
        aux_respostas = dict()
        aux_docID = dict()
        aux_sliders = list()
        
        
        contagem_tmp = 0
        for resultDoc in result.get_result()["results"]:
            
            
            text = resultDoc["text"]
            if( len(text) > 1000 ):  text = text[:1000]
            
            
            aux_respostas[str(contagem_tmp)] = text
            aux_docID[f"dc_id({contagem_tmp})"] = resultDoc["id"]
            
            
            print(text)
            #pagina = encontra_pagina(text)
            #print(text+pagina)
            #print("\n")
                
            
            aux_sliders.append(widgets.IntSlider(
    value=50,
    min=0,
    max=100,
    step=1,
    description='Relevância:',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='d'
            
))
            display(aux_sliders[contagem_tmp])
            contagem_tmp+=1
        
        sliders.append(aux_sliders)
        doc_ids.append(aux_docID)
        respostas.append(aux_respostas)
        botoes.append(widgets.Button(
    description='Salvar notas',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip= f'{n_botao}',
    icon='check' # (FontAwesome names without the `fa-` prefix)
))
        display(botoes[n_botao])
        botoes[n_botao].on_click(callback_botao)

        
    

    
text.on_submit(callback)


# botão pra mostrar tabela, se precisar

# mostrardf = widgets.Button(
#     description='Mostrar tabela',
#     disabled=False,
#     button_style='info', # 'success', 'info', 'warning', 'danger' or ''
#     icon='check' # (FontAwesome names without the `fa-` prefix)
# )
# display(mostrardf)

# def callback_botao2(b):
#     dataframe = pd.read_csv("teste_relevancia.tsv", sep='\t')
#     with out:
#         display(dataframe)
        
# mostrardf.on_click(callback_botao2)




Text(value='', placeholder='')

In [8]:

import PyPDF2



def encontra_pagina(substring):

    object = PyPDF2.PdfFileReader("Guia-de-integralização.pdf")

    NumPages = object.getNumPages()
    #print(f"numero de paginas = {NumPages}")



    for i in range(0, NumPages):
        PageObj = object.getPage(i)
        texto_pagina = PageObj.extractText() 

        if (texto_pagina.find(substring) != -1):
            return(f"\nNa página: {i+1}!")
    return "pagina nao encontrada"

    




In [22]:
#CABEÇALHO DO DATAFRAME




# colunas = ['Perguntas',"dc_id(0)",'0', 'Notas(0)',"dc_id(1)",'1', 'Notas(1)',"dc_id(2)",'2', 'Notas(2)',"dc_id(3)",'3', 'Notas(3)',"dc_id(4)",'4', 'Notas(4)']

# df = pd.DataFrame(columns=colunas,dtype = 'string' )

# df.to_csv("teste_relevancia.tsv", sep="\t",index = False)
# df


,Perguntas,dc_id(0),0,Notas(0),dc_id(1),1,Notas(1),dc_id(2),2,Notas(2),dc_id(3),3,Notas(3),dc_id(4),4,Notas(4)


In [16]:
# dicionario = dict()
# dicionario['Perguntas'] = 'qto custa o pão?'
# dicionario['0'] = '20'
# dicionario['Notas(0)']= '0'
# dicionario['1']= '10'
# dicionario['Notas(1)']= '0'
# dicionario['2']= '10'
# dicionario['Notas(2)']= '10'
# dicionario['3']= '10'
# dicionario['Notas(3)']= '10'
# dicionario['4']= '10'
# dicionario['Notas(4)']= '10'



# df = df.append(dicionario, ignore_index=True)
# df

,Perguntas,dc_id(0),0,Notas(0),dc_id(1),1,Notas(1),dc_id(2),2,Notas(2),dc_id(3),3,Notas(3),dc_id(4),4,Notas(4)
0,qto custa o pão?,NaN,20,0,NaN,10,0,NaN,10,10,NaN,10,10,NaN,10,10


In [21]:
# notas_colunas = [f'Notas({x})' for x in range(0,5)]
# notas_atual = [1,2,3,4,5]


# df.loc[df['Perguntas'] == "qto custa o pão?", notas_colunas] += [f',{x}' for x in notas_atual]

# df


,Perguntas,dc_id(0),0,Notas(0),dc_id(1),1,Notas(1),dc_id(2),2,Notas(2),dc_id(3),3,Notas(3),dc_id(4),4,Notas(4)
0,qto custa o pão?,NaN,20,"0,1,1",NaN,10,"0,1,2",NaN,10,"10,1,3",NaN,10,"10,1,4",NaN,10,"10,1,5"
